### Imports 

In [ ]:
import os

os.chdir('../')

# HTML PARSER
from bs4 import BeautifulSoup as soup

# INITIALIZE DRIVER
from selenium import webdriver

# WAIT AND CONDITIONS METHODS
# available since 2.26.0
from selenium.webdriver.support.ui import Select

# Exceptions
from selenium.common.exceptions import TimeoutException


# METHODS
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from locators import Login, Base, LatMenu, \
    Main, ListaBlocos, Bloco, Processo, Envio


from base import Page

import re

import pandas as pd

In [ ]:
def login_SEI(driver, usr, pwd):
        """
        Esta função recebe um objeto Webdrive e as credenciais  
        do usuário, loga no SEI - ANATEL e retorna uma instância da classe  
        SEI. 
        """

        page = Page(driver)
        page.driver.get(Login.URL)
        page.driver.maximize_window()

        usuario = page.wait_for_element_to_click(Login.LOGIN)
        senha = page.wait_for_element_to_click(Login.SENHA)

        # Clear any clutter on the form
        usuario.clear()
        usuario.send_keys(usr)

        senha.clear()
        senha.send_keys(pwd)

        # Hit Enter
        senha.send_keys(Keys.RETURN)

        return SEI(page.driver)

In [71]:
class SEI(Page):
    """
    Esta subclasse da classe Page define métodos de execução de ações na 
    página principal do SEI e de resgate de informações
    """
    
    processos = []
    
    def ver_proc_detalhado(self):
        """
        Expands the visualization from the main page in SEI
        """
        try:
            ver_todos = self.wait_for_element_to_click(Main.FILTROATRIBUICAO)
            
            if ver_todos.text =="Ver todos os processos":
                ver_todos.click()
        
        except TimeoutException:
            
            print("A página não carregou no tempo limite ou cheque o link\
                  'ver todos os processos'")
            
        try:
            
            visual_detalhado = self.wait_for_element_to_click(Main.TIPOVISUALIZACAO)
        
            if visual_detalhado.text == "Visualização detalhada":
                visual_detalhado.click()
                
        except TimeoutException:
            
            print("A página não carregou no tempo limite ou cheque o link\
            de visualização detalhada")
            
    def isPaginaInicial(self):
        """Retorna True se a página estiver na página inicial do SEI, False
        caso contrário"""        
        return self.get_title() == 'SEI - Controle de Processos'

    def go_to_initial_page(self):
        """
        Navega até a página inicial do SEI caso já esteja nela 
        a página é recarregada
        Assume que o link está presente em qualquer subpágina do SEI
        """
        self.wait_for_element_to_click(
            Base.INITIALPAGE).click()

    def exibir_menu_lateral(self):
        """
        Exibe o Menu Lateral á Esquerda no SEI para acessos aos seus diversos
        links
        Assume que o link está presente em qualquer subpágina do SEI
        """
        menu = self.wait_for_element(Base.EXIBIRMENU)

        if menu.get_attribute("title") == "Exibir Menu do Sistema":
            menu.click()
            
    def itera_processos(self):
        """
        Navega as páginas de processos abertos no SEI e guarda as tags
        html dos processos como objeto soup no atributo processos_abertos
        """     
        # assegura que está inicial
        if not self.isPaginaInicial():
            self.go_to_initial_page()

        # Mostra página com informações detalhadas
        self.ver_proc_detalhado()
        
        contador = Select(self.wait_for_element(Main.CONTADOR))

        pages = [pag.text for pag in contador.options]

        for pag in pages:
            
            # One simple repetition to avoid more complex code
            contador = Select(self.wait_for_element(Main.CONTADOR))
            contador.select_by_visible_text(pag)
            html_sei = soup(self.driver.page_source, "lxml")
            self.processos += html_sei("tr", {"class": 'infraTrClara'})
            
        # percorre a lista de processos
        # cada linha corresponde a uma tag mãe 'tr'
        # substituimos a tag mãe por uma lista das tags filhas 'tag.contents', descartando os '\n'
        # a função lista_to_dict_tags recebe essa lista e retorna um dicionário das tags
        self.processos =  [lista_to_dict_tags([tag for tag in line.contents if tag !='\n']) for line in self.processos]     

In [72]:
def lista_to_dict_tags(lista_tags):

    tags = ['checkbox', 'anotacao', 'situacao', 'marcador', 'aviso', 
            'peticionamento', 'processo', 'atribuicao', 'tipo', 'interessado']
    
    dict_tags = {tag:None for tag in tags}

    assert len(lista_tags) == 6, "Verifique o nº de tags de cada linha do processo, valor diferente de 10"

    dict_tags['checkbox'] = lista_tags[0].find(class_='infraCheckbox')

    controles = lista_tags[1].find_all('a')
    
    for tag_a in controles:
        
        if tag_a.find(src=re.compile("imagens/sei_anotacao")):
            dict_tags['anotacao'] = tag_a
            
        elif tag_a.find(src = re.compile("imagens/sei_situacao")):
            dict_tags['situacao'] = tag_a
            
        elif tag_a.find(src= re.compile("imagens/marcador")):
            dict_tags['marcador'] = tag_a
            
        elif tag_a.find(src = re.compile("imagens/exclamacao")):
            dict_tags['aviso'] = tag_a
            
        elif tag_a.find(src = re.compile("imagens/peticionamento")):
            dict_tags['peticionamento'] = tag_a
            
    try:    
        dict_tags['processo'] = lista_tags[2].find('a')       
    except:        
        #print("Problema na tag 'processo'")
        
    try:
        dict_tags['atribuicao'] = lista_tags[3].find('a')
    except:
        #print("Problema na tag ou processo sem atribuição")
        
    try:    
        dict_tags['tipo'] = lista_tags[4].string
    except:
        #print("Problema na tag 'tipo de processo'")
    
    try:        
        #dict_tags['interessado'] = lista_tags[5].find(class_='spanItemCelula').string        
    except:        
        #print("Problema na tag ou processo sem 'interessado")

    return dict_tags
    
    


In [ ]:
driver = webdriver.Chrome()
sei = login_SEI(driver, 'rsilva', 'Savorthemom3nts')

In [73]:
def dict_to_df(dict_tags):
    
     tags = ['anotacao', 'situacao', 'marcador', 'aviso', 
            'peticionamento', 'processo', 'atribuicao', 'tipo', 'interessado']
    
    dict_tags = {tag:None for tag in tags}
    
    if dict_tags[]     
    
    
        

sei = SEI(sei.driver)

In [110]:
sei.processos[25]['anotacao'].attrs['onmouseover']

"return infraTooltipMostrar('Analisado','rodrigoferraz');"

In [85]:
re.compile()

In [86]:
tipos_sem_atribuicao

[{'anotacao': None,
  'atribuicao': None,
  'aviso': None,
  'checkbox': <input class="infraCheckbox" id="chkDetalhadoItem47" name="chkDetalhadoItem47" onclick="infraSelecionarItens(this,'Detalhado');" title="53504.203989/2015-91" type="checkbox" value="214778"/>,
  'interessado': 'Marcelo Vaz Netto',
  'marcador': None,
  'peticionamento': None,
  'processo': <a class="processoVisualizado" href="controlador.php?acao=procedimento_trabalhar&amp;acao_origem=procedimento_controlar&amp;acao_retorno=procedimento_controlar&amp;id_procedimento=214778&amp;infra_sistema=100000100&amp;infra_unidade_atual=110001005&amp;infra_hash=c5d3150ae9840e6dadc257aa05423491f33f4535ecf5f0750c6a87061e483181" onmouseout="return infraTooltipOcultar();" onmouseover="return infraTooltipMostrar('Requisição de Afastamento  Curso de Pós-Graduação','Pessoal: Afastamento para Pós-Graduação');">53504.203989/2015-91</a>,
  'situacao': None,
  'tipo': 'Pessoal: Afastamento para Pós-Graduação'}]

In [ ]:
sei.processos[2].contents